<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/mrc_LSTM_baseline0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Import Libraries and Read Data

In [92]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
project_path = "/content/drive/My Drive/AIML-MRC-Capstone/datasets/Squad2.0/TrainingDataset/"

squad_df = pd.read_csv(project_path+'squad_data_final.csv')
squad_df.drop('Unnamed: 0',axis=1,inplace=True)
squad_df.tail(5)

,title,context,question,id,answer_start,answer,plausible_answer_start,plausible_answer,is_impossible,clean_context,clean_question,clean_answer
130301,Matter,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,5a7e070b70df9f001a875439,NaN,NaN,485.0,matter,True,term matter used throughout physics bewilderin...,physics has broadly agreed on the definition o...,IMPOSSIBLE
130302,Matter,"The term ""matter"" is used throughout physics i...",Who coined the term partonic matter?,5a7e070b70df9f001a87543a,NaN,NaN,327.0,Alfvén,True,term matter used throughout physics bewilderin...,who coined the term partonic matter,IMPOSSIBLE
130303,Matter,"The term ""matter"" is used throughout physics i...",What is another name for anti-matter?,5a7e070b70df9f001a87543b,NaN,NaN,350.0,Gk. common matter,True,term matter used throughout physics bewilderin...,what is another name for antimatter,IMPOSSIBLE
130304,Matter,"The term ""matter"" is used throughout physics i...",Matter usually does not need to be used in con...,5a7e070b70df9f001a87543c,NaN,NaN,529.0,a specifying modifier,True,term matter used throughout physics bewilderin...,matter usually does not need to be used in con...,IMPOSSIBLE
130305,Matter,"The term ""matter"" is used throughout physics i...",What field of study has a variety of unusual c...,5a7e070b70df9f001a87543d,NaN,NaN,37.0,physics,True,term matter used throughout physics bewilderin...,what field of study has a variety of unusual c...,IMPOSSIBLE


# Lets use 2 LSTM network for context and question and use a concat layer to merge. y is the answer 

# Step 1 Tokenize and Vectorize

### Tokenizing context and question

In [0]:
context_tokenize = preprocessing.text.Tokenizer()
context_tokenize.fit_on_texts(squad_df['clean_context']) #Fit it on clean_context

questions_tokenize= preprocessing.text.Tokenizer()
questions_tokenize.fit_on_texts(squad_df['clean_question'])

In [0]:
#convert the context text to indexes
context_sequence= context_tokenize.texts_to_sequences(squad_df['clean_context'])
# convert the questions to indexes
questions_sequence= questions_tokenize.texts_to_sequences(squad_df['clean_question'])

In [101]:
print(squad_df['clean_context'][3000])
print(context_sequence[3000])
print(squad_df['clean_question'][2000])
print(questions_sequence[2000])

office buildings shanghais financial district including jin mao tower hong kong new world tower evacuated receptionist tibet hotel chengdu said things calm hotel evacuated guests meanwhile workers ford plant sichuan evacuated 10 minutes chengdu shuangliu international airport shut control tower regional radar control evacuated one silkair flight diverted landed kunming result cathay pacific delayed legs quadruple daily hong kong london route due disruption air traffic services chengdu shuangliu airport reopened later evening may 12 offering limited service airport began used staging area relief operations
[360, 464, 56909, 483, 454, 21, 7582, 7198, 1401, 2632, 2957, 4, 14, 1401, 5514, 56910, 1051, 2806, 4752, 249, 1260, 9282, 2806, 5514, 7725, 2752, 1600, 3891, 692, 1892, 5514, 204, 2129, 4752, 26680, 78, 578, 5194, 118, 1401, 618, 5590, 118, 5514, 2, 56911, 1482, 11598, 4122, 16977, 229, 56912, 962, 3628, 4968, 23304, 1030, 2632, 2957, 207, 1339, 92, 6491, 162, 1512, 337, 4752, 26680,

### Find Max Sequence Length for both the Sequences

In [99]:
# max length of context
max_context_seq_length= max(len(txt) for txt in context_sequence)
print('max_context_seq_length=',max_context_seq_length)

# vocab size of context
context_vocab_size=len(context_tokenize.word_index)
print('context_vocab_size=',context_vocab_size)

max_question_seq_length=max(len(txt) for txt in questions_sequence)
print('max_question_seq_length=',max_question_seq_length)

# vocab size of questions
questions_vocab_size=len(questions_tokenize.word_index)
print('questions_vocab_size=',questions_vocab_size)

max_context_seq_length= 426
context_vocab_size= 93529
max_question_seq_length= 40
questions_vocab_size= 47289


### Padding the sequences

In [0]:
# padding context
context_input_data= tf.keras.preprocessing.sequence.pad_sequences(context_sequence, maxlen=max_context_seq_length, padding='pre')
# padding question
question_input_data=tf.keras.preprocessing.sequence.pad_sequences(questions_sequence, maxlen=max_question_seq_length,padding='pre')

# Build the LSTM Model for both Sequence

In [0]:
embedding_size = 50
rnn_units=100

### LSTM for Context and Question

In [104]:
# CONTEXT LSTM
# input layer
context_input=layers.Input(shape=(max_context_seq_length,))
# Build Embedding layer and Get Embedding Layer output
context_embedding_output=layers.Embedding(context_vocab_size+1, embedding_size)(context_input)
#LSTM layer and its output
CONTEXT_LSTM= layers.LSTM(rnn_units,name='CONTEXT_LSTM')(context_embedding_output)

#build a list to feed for concatenation
#context_states= [state_h1, state_c1]

# QUESTION LSTM
#input layer
question_input=layers.Input(shape=(max_question_seq_length,))
#Embedding layer and #Embedding layer output
question_embedding_output=layers.Embedding(input_dim=questions_vocab_size+1, output_dim=embedding_size)(question_input)
#LSTM2 layer 
QUESTION_LSTM= tf.keras.layers.LSTM(rnn_units,name='QUESTION_LSTM')(question_embedding_output)

print(CONTEXT_LSTM)
print(QUESTION_LSTM)

Tensor("CONTEXT_LSTM/Identity:0", shape=(None, 256), dtype=float32)
Tensor("QUESTION_LSTM/Identity:0", shape=(None, 256), dtype=float32)


# Concat the two LSTM layers

In [0]:
MERGED=layers.concatenate([CONTEXT_LSTM,QUESTION_LSTM])

# Create Decoder for Answer

# Add  Start and  End tokens to Answers

In [0]:
squad_df['answer_start_end']= '<start>' + squad_df['clean_answer'] + '<end>'


In [0]:
squad_df['answer_start_end']=squad_df['answer_start_end'].astype(str)


#Tokenize the Answers

In [0]:
answers_tokenize=tf.keras.preprocessing.text.Tokenizer()
answers_tokenize.fit_on_texts(squad_df['answer_start_end'])

In [0]:
#Vocab
len(answers_tokenize.word_index)

41475

In [0]:
print(answers_tokenize.word_index)

{'end': 1, 'start': 2, 'impossible': 3, 'the': 4, 'of': 5, 'and': 6, 'a': 7, 'to': 8, 'in': 9, 'or': 10, 'for': 11, 'million': 12, 'as': 13, 'by': 14, 'on': 15, 'is': 16, 'with': 17, 'from': 18, 'that': 19, 'an': 20, 'new': 21, 'century': 22, 'two': 23, 'their': 24, 'one': 25, 'was': 26, 'united': 27, '1': 28, 'at': 29, 'not': 30, 'be': 31, 'years': 32, 'more': 33, 'are': 34, 'three': 35, '2': 36, 'it': 37, 'states': 38, 'war': 39, '5': 40, 'over': 41, 'world': 42, 'than': 43, 'its': 44, '3': 45, 'his': 46, 'state': 47, 'four': 48, 'national': 49, 'john': 50, 'may': 51, 'city': 52, 'between': 53, 'all': 54, 'de': 55, 'were': 56, 'system': 57, 'other': 58, 'university': 59, 'british': 60, 'government': 61, 'billion': 62, 'they': 63, '4': 64, 'five': 65, '8': 66, 'first': 67, 'north': 68, 'bc': 69, '6': 70, 'have': 71, 'international': 72, 'no': 73, 'south': 74, 'general': 75, '7': 76, 'law': 77, 'power': 78, 'french': 79, '10': 80, 'east': 81, 'early': 82, 'high': 83, 'which': 84, '20':

In [0]:
answers_seq = answers_tokenize.texts_to_sequences(squad_df['answer_start_end']) #Convert sentences to numbers 

In [0]:
answers_seq[2000]

[2, 924, 7781, 1]

# Get maximum length and pad the sequences

In [0]:
max_answers_seq_length=max(len(txt) for txt in squad_df['answer_start_end'])
print(max_answers_seq_length)

answers_vocab_size=len(answers_tokenize.word_index)
print(answers_vocab_size)

248
41475


In [0]:
answers_input_data= tf.keras.preprocessing.sequence.pad_sequences(answers_seq,maxlen=max_answers_seq_length,padding='pre')

# Building Decoder Output

In [0]:
answers_input_data.shape

(130306, 248)

In [0]:
#Initialize array
answers_target_data = np.zeros((answers_input_data.shape[0], #number of sentences 130306
                                answers_input_data.shape[1])) #number of words in each sentence 248

#Shift Target output by one word
for i in range(answers_input_data.shape[0]):
    for j in range(1,answers_input_data.shape[1]):
        answers_target_data[i][j-1] = answers_input_data[i][j]

In [0]:
squad_df['answer_start_end'][0]

'<start>in the late 1990s<end>'

In [0]:
answers_input_data[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [0]:
answers_target_data[0]

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   

# Convert Answers to one-hot vector

In [0]:
#answers_target_data_one_hot= np.zeros((answers_input_data.shape[0], 
                                       answers_input_data.shape[1],len(answers_tokenize.word_index)+1))

In [0]:
answers_embedding_size = 50
rnn_units = 256

# Build Decoder

In [0]:
#input layer
answers_inputs=tf.keras.layers.Input(shape=(None,))

#Embedding
answers_embedding=tf.keras.layers.Embedding(answers_vocab_size+1, answers_embedding_size)
answers_embedding_output=answers_embedding(answers_inputs)

#lstm layer
answers_lstm= tf.keras.layers.LSTM(rnn_units,return_sequences=True, return_state=True)

#LSTM Output, State initialization from Encoder states(concat of question and answer)
#Output will be all hidden sequences, last 'h' state and last 'c' state

lstm3=answers_lstm(answers_embedding_output,initial_state=<concatlayer h and c)

#dense layer
lstm3_dense= tf.keras.layers.Dense(answers_vocab_size+1,activation='softmax',)

#answer output
answer_outputs=lstm3_dense(lstm3)

# Build Model using Encoder ( output of concat) and Decoder

# Train the Model